In [1]:
import requests
import datetime
from datetime import datetime as dt
import patoolib
import os
import pandas as pd
import sqlalchemy 
import psycopg2
from sqlalchemy import create_engine
import numpy as np
from datetime import timedelta
import geopandas as gpd
from mapbox import Uploader
import sys

In [2]:
sys.path.append('..')

In [3]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])
mapbox_api=os.environ['MAPBOX_API_KEY']
mapbox_up=os.environ['MAPBOX_API_UPLOAD_KEY']

In [4]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [5]:
url='https://matiascerda.carto.com/api/v2/sql?format=CSV&filename=datos-movilidad-isci.csv&q=select%20*%20from%20data_variacion'

In [6]:
sql="select * from geo.zonas_censales"
zonas= gpd.GeoDataFrame.from_postgis(sql,con=cnx, geom_col='geometry')
zonas['GEOCODIGO']=zonas['GEOCODIGO'].astype(str).str.strip()

In [9]:
zonas['PERSONAS_6_A_64_AÑOS']=zonas['PERSONAS_6_A_14_AÑOS']+zonas['PERSONAS_15_A_64_AÑOS']

In [10]:
crs=zonas.crs

In [11]:
#diferencias entre entradas y salidas de zonas censales estimadas por ISCI-entel Ocean
ISCIentel=pd.read_csv(url,encoding='utf-8')
#diferencias entre entradas
ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']]=ISCIentel['dif_entrada'].str.replace('[','').str.replace(',]','').str.replace('%','').str.replace(']','').str.split(',',expand=True)
ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']]=ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']].astype(int)
#diferencias entre salidas
ISCIentel[['dif_salida_lim_inf','dif_salida_lim_sup']]=ISCIentel['dif_salida'].str.replace('[','').str.replace(',]','').str.replace('%','').str.replace(']','').str.split(',',expand=True)
ISCIentel[['dif_salida_lim_inf','dif_salida_lim_sup']]=ISCIentel[['dif_salida_lim_inf','dif_salida_lim_sup']].astype(int)
ISCIentel['zona_censal']=ISCIentel['zona_censal'].astype(str).str.strip()
ISCIentel=ISCIentel.drop(['the_geom','cartodb_id','the_geom_webmercator'],axis=1)
ISCIentel=ISCIentel.merge(zonas,right_on='GEOCODIGO',left_on='zona_censal',how='left')
ISCIentel['date']=pd.to_datetime('2020-'+ISCIentel['week'].astype(str)+'-6',format='%Y-%W-%w')
ISCIentel['salidas']=(ISCIentel['dif_salida_lim_sup']-ISCIentel['dif_salida_lim_inf'])/2+ISCIentel['dif_salida_lim_inf']
ISCIentel['PERSONAS_MOVILIZADAS']=ISCIentel['PERSONAS_6_A_64_AÑOS']+ISCIentel['PERSONAS_6_A_64_AÑOS']/100*ISCIentel['salidas']

<ipython-input-11-39a840ff3a8b>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']]=ISCIentel['dif_entrada'].str.replace('[','').str.replace(',]','').str.replace('%','').str.replace(']','').str.split(',',expand=True)
<ipython-input-11-39a840ff3a8b>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  ISCIentel[['dif_entrada_lim_inf','dif_entrada_lim_sup']]=ISCIentel['dif_entrada'].str.replace('[','').str.replace(',]','').str.replace('%','').str.replace(']','').str.split(',',expand=True)
<ipython-input-11-39a840ff3a8b>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ISCIentel[['d

In [12]:
#Variabilidad PONDERADA
pop_comuna=pd.DataFrame(ISCIentel.groupby(['week','comuna'])['PERSONAS_6_A_64_AÑOS'].sum())
pop_comuna=pd.DataFrame(pop_comuna.reset_index().groupby('comuna')['PERSONAS_6_A_64_AÑOS'].mean()).reset_index()
pop_comuna=pop_comuna.rename(columns={'PERSONAS_6_A_64_AÑOS':'PERSONAS_6_A_64_AÑOS_COMUNA'})
pop_comuna=pop_comuna.reset_index()
#ISCIentel['PERSONAS_15_A_64_AÑOS_COMUNA']
ISCIentel=ISCIentel.merge(pop_comuna,on='comuna',how='left')
#% de personas 15 a 64 años comunal por zona censal
ISCIentel['PERSONAS_6_A_64_AÑOS_FACTOR']=ISCIentel['PERSONAS_6_A_64_AÑOS']/ISCIentel['PERSONAS_6_A_64_AÑOS_COMUNA']
ISCIentel['salidas_comunal_ponderada']=ISCIentel['salidas']*ISCIentel['PERSONAS_6_A_64_AÑOS_FACTOR']

In [13]:
ISCIentel = gpd.GeoDataFrame(ISCIentel)
ISCIentel.set_crs(crs)

,zona_censal,comuna,region,week,dif_salida,dif_entrada,dif_entrada_lim_inf,dif_entrada_lim_sup,dif_salida_lim_inf,dif_salida_lim_sup,...,PERSONAS_15_A_64_AÑOS,PERSONAS_MAS_65_AÑOS,PERSONAS_6_A_64_AÑOS,date,salidas,PERSONAS_MOVILIZADAS,index,PERSONAS_6_A_64_AÑOS_COMUNA,PERSONAS_6_A_64_AÑOS_FACTOR,salidas_comunal_ponderada
0,16109011002,YUNGAY,REGIÓN DE ÑUBLE,36,"[-40%,-21%,]","[+61%,+100%,]",61,100,-40,-21,...,2738.0,585.0,3105.0,2020-09-12,-30.5,2157.975,315.0,10143.0,0.306122,-9.336735
1,4106011001,VICUÑA,REGIÓN DE COQUIMBO,31,"[-40%,-21%,]","[-21%,+20%,]",-21,20,-40,-21,...,2626.0,355.0,3166.0,2020-08-08,-30.5,2200.370,306.0,13188.0,0.240067,-7.322035
2,13119161003,MAIPÚ,REGIÓN METROPOLITANA DE SANTIAGO,42,"[-40%,-21%,]","[+41%,+60%,]",41,60,-40,-21,...,3870.0,474.0,4290.0,2020-10-24,-30.5,2981.550,151.0,424316.0,0.010110,-0.308367
3,7304021005,MOLINA,REGIÓN DEL MAULE,9,"[-21%,+20%,]","[-21%,+20%,]",-21,20,-21,20,...,2001.0,346.0,2267.0,2020-03-07,-0.5,2255.665,162.0,29234.0,0.077547,-0.038773
4,4101031003,LA SERENA,REGIÓN DE COQUIMBO,30,"[-40%,-21%,]","[-40%,-21%,]",-40,-21,-40,-21,...,3292.0,198.0,3575.0,2020-08-01,-30.5,2484.625,117.0,157671.0,0.022674,-0.691551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214055,13127041004,RECOLETA,REGIÓN METROPOLITANA DE SANTIAGO,48,"[-21%,+20%,]","[+61%,+100%,]",61,100,-21,20,...,1853.0,383.0,2080.0,2020-12-05,-0.5,2069.600,242.0,124824.0,0.016663,-0.008332
214056,13127041004,RECOLETA,REGIÓN METROPOLITANA DE SANTIAGO,49,"[-21%,+20%,]","[+61%,+100%,]",61,100,-21,20,...,1853.0,383.0,2080.0,2020-12-12,-0.5,2069.600,242.0,124824.0,0.016663,-0.008332
214057,13127041004,RECOLETA,REGIÓN METROPOLITANA DE SANTIAGO,50,"[-21%,+20%,]","[+61%,+100%,]",61,100,-21,20,...,1853.0,383.0,2080.0,2020-12-19,-0.5,2069.600,242.0,124824.0,0.016663,-0.008332
214058,13127041004,RECOLETA,REGIÓN METROPOLITANA DE SANTIAGO,51,"[-21%,+20%,]","[+61%,+100%,]",61,100,-21,20,...,1853.0,383.0,2080.0,2020-12-26,-0.5,2069.600,242.0,124824.0,0.016663,-0.008332


In [14]:
name='isci_entel'
schema='movilidad'
if not cnx.dialect.has_schema(cnx, schema):
    print('schema '+schema+' does not exist, creating it')
    cnx.execute(sqlalchemy.schema.CreateSchema(schema))
else:
    print('schema '+schema+' exists, will not be created')

schema movilidad exists, will not be created


In [15]:
ISCIentel['geometry']=ISCIentel['geometry'].centroid
ISCIentel['Lon']=ISCIentel['geometry'].x
ISCIentel['Lat']=ISCIentel['geometry'].y
ISCIentel=ISCIentel.drop('geometry',axis=1)
ISCIentel=pd.DataFrame(ISCIentel)

<ipython-input-15-f461c9b017e0>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ISCIentel['geometry']=ISCIentel['geometry'].centroid


In [16]:
#for r in ISCIentel['REGION'].unique():
#    name='isci_entel'+str(r)
#    print('saving table '+name)
    #ISCIentel[ISCIentel['REGION']==r].to_postgis(name=name+'_'+str(r), schema=schema,con=cnx,if_exists='replace')
#    ISCIentel[ISCIentel['REGION']==r].to_sql(name, schema=schema,con=cnx,if_exists='replace')

In [17]:
#geojson=ISCIentel[ISCIentel['REGION']=='13']
#geojson['date']=geojson['date'].astype(str)
#geojson=geojson.to_json()

In [18]:
#ISCIentel[ISCIentel['REGION']=='13'].to_file("../ETLcache/mov_temp.geojson", driver='GeoJSON')

In [19]:
#with open("../ETLcache/mov_temp.geojson", 'rb') as src:
#    upload_resp = service.upload(src, 'test_dataset')
#upload_resp
#upload_id = upload_resp.json()['id']

In [20]:
ISCIentel.to_sql('isci_entel', schema=schema,con=cnx,if_exists='replace')

In [21]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_movilidad.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/movilidad.ipynb')

0